In [54]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
idx = pd.IndexSlice

In [160]:
semaforo = pd.read_csv('g_semaforo_2.csv')\
    .drop_duplicates(subset=['cve_mun', 'categoria'])\
    .set_index(['cve_mun', 'categoria'])
# datos para mapa
#semaforo.query('categoria=="moda"')[['valor']].droplevel('categoria').to_csv('semaforo_mapa.csv')
semaforo.head()

valor      cve   original
cve_mun categoria                             
1001    sem_total      2.0    total  -4.031691
        sem_mujeres    2.0  mujeres  -3.395323
        sem_jovenes    1.0  jovenes -10.075730
        sem_mayores    2.0  mayores  -2.989917
        sem_uma        2.0      uma  -3.972211

In [152]:
crecimiento_wide = semaforo['original'].unstack('categoria')
crecimiento_wide.head()

categoria,moda,sem_jovenes,sem_mayores,sem_mujeres,sem_pyme,sem_total,sem_uma
cve_mun,,,,,,,
1001,NaN,-10.075730,-2.989917,-3.395323,-0.618419,-4.031691,-3.972211
1002,NaN,9.375000,7.653061,6.792453,8.742004,0.713649,5.263158
1003,NaN,-5.746141,3.908046,2.847806,-5.720824,-2.774032,-2.959566
1004,NaN,-13.225806,4.794521,3.936039,-14.545455,-2.111801,-1.955671
1006,NaN,4.600484,2.666667,4.653802,7.493703,2.482495,4.388610


In [141]:
# Municipios que tienen menos empleados que a inicio de la pandemia
crecimiento_wide.lt(0).mean()

categoria
moda           0.000000
sem_jovenes    0.499497
sem_mayores    0.401408
sem_mujeres    0.433099
sem_pyme       0.464789
sem_total      0.496982
sem_uma        0.478873
dtype: float64

## Número de empleos totales que faltan para recuperarse

In [136]:
municipios_empleos_totales = crecimiento_wide.query('sem_total<0').index.tolist()
pd.read_csv('imss/datos/semaforo/asegurados_total_202008.csv')\
    .query('cve_mun==@municipios_empleos_totales & year==2020 & mes==[2, 8]')\
    .groupby(['cve_mun', 'mes'])[['asegurados']].sum()\
    .groupby('cve_mun').diff()['asegurados'].sum()

-1092974.0

## Número de empleos de mujeres que faltan para recuperarse 

In [145]:
municipios_empleos_mujeres = crecimiento_wide.query('sem_mujeres<0').index.tolist()
pd.read_csv('imss/datos/semaforo/asegurados_x_sexo_202008.csv')\
    .query('sexo=="Mujeres" & cve_mun==@municipios_empleos_mujeres & year==2020 & mes==[2, 8]')\
    .groupby(['cve_mun', 'mes'])[['asegurados']].sum()\
    .groupby('cve_mun').diff()['asegurados'].sum()

-389834.0

## Número de empleos de jóvenes que faltan para recuperarse 

In [148]:
municipios_empleos_jovenes = crecimiento_wide.query('sem_jovenes<0').index.tolist()
pd.read_csv('imss/datos/semaforo/asegurados_x_edad_202008.csv')\
    .query('categoria_edad=="Jóvenes" & cve_mun==@municipios_empleos_jovenes & year==2020 & mes==[2, 8]')\
    .groupby(['cve_mun', 'mes'])[['asegurados']].sum()\
    .groupby('cve_mun').diff()['asegurados'].sum()

-359249.0

## Número de empleos de adultos mayores que faltan para recuperarse 

In [151]:
municipios_empleos_mayores = crecimiento_wide.query('sem_mayores<0').index.tolist()
pd.read_csv('imss/datos/semaforo/asegurados_x_edad_202008.csv')\
    .query('categoria_edad=="Mayores" & cve_mun==@municipios_empleos_mayores & year==2020 & mes==[2, 8]')\
    .groupby(['cve_mun', 'mes'])[['asegurados']].sum()\
    .groupby('cve_mun').diff()['asegurados'].sum()

-78925.0

## Número de empleos en pequeñas empresas que faltan para recuperarse 

In [154]:
municipios_empleos_pyme = crecimiento_wide.query('sem_pyme<0').index.tolist()
pd.read_csv('imss/datos/semaforo/asegurados_x_edad_202008.csv')\
    .query('cve_mun==@municipios_empleos_pyme & year==2020 & mes==[2, 8]')\
    .groupby(['cve_mun', 'mes'])[['asegurados']].sum()\
    .groupby('cve_mun').diff()['asegurados'].sum()

-358557.0

## Número de empleos de bajos ingresos que faltan para recuperarse 

In [156]:
municipios_empleos_pyme = crecimiento_wide.query('sem_pyme<0').index.tolist()
pd.read_csv('imss/datos/semaforo/asegurados_uma50_202008.csv')\
    .query('cve_mun==@municipios_empleos_pyme & year==2020 & mes==[2, 8]')\
    .groupby(['cve_mun', 'mes'])[['asegurados']].sum()\
    .groupby('cve_mun').diff()['asegurados'].sum()

-803102.0

## Semaforo por municipio

In [161]:
colores_semaforo = {1: 'Rojo', 2: 'Amarillo', 3: 'Verde'}
semaforo.groupby('categoria')['valor'].value_counts().unstack('valor').rename(columns=colores_semaforo)

valor,Rojo,Amarillo,Verde
categoria,,,
moda,464,683,787
sem_jovenes,697,456,555
sem_mayores,366,677,686
sem_mujeres,348,747,707
sem_pyme,370,686,870
sem_total,415,689,827
sem_uma,441,637,849
